In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

df = pd.read_csv("../data/machine_failure_prediction_using_sensor_data.csv")

# Descripción de las columnas
# footfall: Número de personas u objetos que pasan por delante de la máquina.
# tempMode: Modo o ajuste de temperatura de la máquina.
# AQ: Índice de calidad del aire cerca de la máquina.
# USS: Datos del sensor ultrasónico, que indican mediciones de proximidad.
# CS: Lecturas actuales del sensor, que indican el consumo eléctrico de la máquina.
# VOC: Nivel de compuestos orgánicos volátiles detectado cerca de la máquina.
# RP: Posición rotacional o RPM (revoluciones por minuto) de las piezas de la máquina.
# IP: Presión de entrada a la máquina.
# Temperatura: Temperatura de funcionamiento de la máquina.
# Fallo: Indicador binario de fallo de la máquina (1 para fallo, 0 para sin fallo).

In [ ]:
# ========================================================
# Visualizar datos
# ========================================================
print("Primeras filas:")
display(df.head())
print("\nInformación general:")
df.info()

In [ ]:
# ========================================================
# Limpieza de datos
# ========================================================
# Se elimina columnas que no vamos a utilizar
if 'Machine_ID' in df.columns:
    df.drop('Machine_ID', axis=1, inplace=True)

# Manejo de valores nulos (NaN)
print(df.isnull().sum())
df.dropna(inplace=True)

# Se hace el cast de los valores a numericos
cols_to_plot = df.select_dtypes(include=['float64', 'int64']).columns[:9]

In [ ]:
# ========================================================
# Analisis exploratorio de datos (EDA)
# ========================================================
print("\n" + "="*80)
print("ANÁLISIS EXPLORATORIO DE DATOS (EDA)")
print("="*80)

print("\n--- RESUMEN ESTADÍSTICO ---")
print(df.describe().round(3))

print("\n Matriz de Correlación:")
correlation = df.corr().round(3)
print(correlation)

plt.figure(figsize=(8, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0,
            vmin=-1, vmax=1, square=True, linewidths=1, fmt=".3f")
plt.title('Matriz de Correlación - Problema de Regresión', fontsize=14, fontweight='bold')
plt.tight_layout()

plt.show()

print("\n INTERPRETACIÓN:")
print(f"   • Correlación VOC-Fail: {correlation.loc['VOC', 'fail']:.3f}")
print(f"   • Correlación AQ-VOC: {correlation.loc['AQ', 'VOC']:.3f}")
print(f"   • Correlación AQ-Fail: {correlation.loc['AQ', 'fail']:.3f}")
print(f"   → Al aumentar el nivel de compuestos orgánicos volátiles detectado cerca de la máquina y aumentan los indice de calidad del aire cerca de la máquina es cuando tiende a fallar")

In [ ]:
# ========================================================
# Visualizaciones
# ========================================================
fig, axes = plt.subplots(1, 9, figsize=(22, 5))

for i, col in enumerate(cols_to_plot):
    sns.histplot(df[col], kde=True, ax=axes[i], color='skyblue')
    axes[i].set_title(col, fontsize=10)
    axes[i].set_xlabel('')
    axes[i].set_ylabel('')

plt.tight_layout()
plt.show()

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Gráfico 1: VOC vs AQ (Dispersión)
# Agregamos comillas a 'VOC', 'AQ' y 'fail'
axes[0].scatter(df['VOC'], df['AQ'], alpha=0.6, c=df['fail'],
                cmap='coolwarm', s=50, edgecolors='k', linewidth=0.5)
axes[0].set_xlabel('Compuestos Orgánicos (VOC)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Calidad de Aire (AQ)', fontsize=12, fontweight='bold')
axes[0].set_title('Calidad de Aire vs Compuestos Orgánicos\n(Color: Fallo)', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Distribución de VOC
# Agregamos comillas a 'VOC'
axes[1].hist(df['VOC'], bins=30, color='indianred', edgecolor='black', alpha=0.7)
axes[1].axvline(df['VOC'].mean(), color='blue', linestyle='--',
                linewidth=2, label=f'Media: {df["VOC"].mean():.2f}')
axes[1].set_xlabel('Compuestos Orgánicos (VOC)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
axes[1].set_title('Distribución de Compuestos Orgánicos\n(Máxima Correlación)', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Gráfico 3: Boxplot de VOC por Estado de Fallo
# Agregamos comillas a 'fail' y 'VOC'
axes[2].boxplot([df[df['fail']==0]['VOC'],
                 df[df['fail']==1]['VOC']],
                labels=['Operación Normal', 'Incidencia (Fail)'],
                patch_artist=True,
                boxprops=dict(facecolor='lightcoral', alpha=0.6))
axes[2].set_ylabel('Compuestos Orgánicos (VOC)', fontsize=12, fontweight='bold')
axes[2].set_title('Compuestos Orgánicos por Estado\nSeparación de Clases', fontsize=13, fontweight='bold')
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# ========================================================
# Preparación de datos
# ========================================================
print("\n" + "="*80)
print("PREPARACIÓN DE DATOS")
print("="*80)

# Separar features y target
X = df[['VOC', 'AQ']].values
y = df['fail'].values

# Escalar features
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# División train-test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print(f"\n División completada:")
print(f"   • Entrenamiento: {len(X_train)} muestras")
print(f"   • Prueba: {len(X_test)} muestras")

In [ ]:
# ========================================================
# Implementación de Regresión Logística
# ========================================================
print("\n" + "="*50)
print("MODELO: REGRESIÓN LOGÍSTICA")
print("="*50)

# Entrenamiento del modelo con datos escalados
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

# Predicciones
y_pred_lr = lr.predict(X_test)

# Evaluación
print("\nMatriz de Confusión (Regresión Logística):")
conf_matrix_lr = confusion_matrix(y_test, y_pred_lr)
print(conf_matrix_lr)

# Visualización de la matriz de confusión adaptada al dataset de fallos
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_lr, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Fallo', 'Fallo'],
            yticklabels=['No Fallo', 'Fallo'])
plt.xlabel('Predicción del Modelo')
plt.ylabel('Estado Real de la Máquina')
plt.title('Matriz de Confusión - Diagnóstico de Incidencias')
plt.show()

# Métricas
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print(f"\n RESULTADOS:")
print(f"   • MSE (Error Cuadrático Medio): {mse_lr:,.2f}")
print(f"   • RMSE (Raíz del MSE): {rmse_lr:,.2f}")
print(f"   • MAE (Error Absoluto Medio): {mae_lr:,.2f}")
print(f"   • R² Score: {r2_lr:.4f} ({r2_lr*100:.2f}% de varianza explicada)")

print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_lr))

# Coeficientes del modelo asociados a los sensores
print("\nInterpretación de Sensores (Coeficientes):")
intercept = lr.intercept_[0]
print(f"Intercepto (Sesgo): {intercept:.4f}")

In [ ]:
# ========================================================
# Implementación de Árbol de Decisión
# ========================================================
print("\n" + "="*50)
print("MODELO: ÁRBOL DE DECISIÓN")
print("="*50)

# Entrenamiento del modelo
tree_model = DecisionTreeClassifier(max_depth=4, random_state=42)
tree_model.fit(X_train, y_train)

# Predicciones
y_pred_tree = tree_model.predict(X_test)

# Evaluación
print("\nMatriz de Confusión (Árbol de Decisión):")
conf_matrix_tree = confusion_matrix(y_test, y_pred_tree)
print(conf_matrix_tree)

# Visualización de la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_tree, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Falla', 'Falla'],
            yticklabels=['No Falla', 'Falla'])
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión - Árbol de Decisión')
plt.savefig('confusion_matrix_tree.png')
plt.show()

# Métricas
mse_tree = mean_squared_error(y_test, y_pred_tree)
rmse_tree = np.sqrt(mse_tree)
mae_tree = mean_absolute_error(y_test, y_pred_tree)
r2_tree = r2_score(y_test, y_pred_tree)

print(f"\n RESULTADOS:")
print(f"   • MSE (Error Cuadrático Medio): {mse_tree:,.2f}")
print(f"   • RMSE (Raíz del MSE): {rmse_tree:,.2f}")
print(f"   • MAE (Error Absoluto Medio): {mae_tree:,.2f}")
print(f"   • R² Score: {r2_tree:.4f} ({r2_tree*100:.2f}% de varianza explicada)")

print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_tree))

In [ ]:
# ========================================================
# Implementación de SVM (Support Vector Machine)
# ========================================================
print("\n" + "="*50)
print("MODELO SUPPORT VECTOR MACHINE (SVM)")
print("="*50)

# Entrenamiento del modelo
svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm.fit(X_train, y_train)

# Predicciones
y_pred_svm = svm.predict(X_test)

# Evaluación
print("\nMatriz de Confusión (SVM):")
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
print(conf_matrix_svm)

# Visualización de la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_svm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Falla', 'Falla'],
            yticklabels=['No Falla', 'Falla'])
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión - SVM')
plt.savefig('confusion_matrix_svm.png')
plt.show()

# Métricas
mse_svm = mean_squared_error(y_test, y_pred_svm)
rmse_svm = np.sqrt(mse_svm)
mae_svm = mean_absolute_error(y_test, y_pred_svm)
r2_svm = r2_score(y_test, y_pred_svm)

print(f"\n RESULTADOS:")
print(f"   • MSE (Error Cuadrático Medio): {mse_svm:,.2f}")
print(f"   • RMSE (Raíz del MSE): {rmse_svm:,.2f}")
print(f"   • MAE (Error Absoluto Medio): {mae_svm:,.2f}")
print(f"   • R² Score: {r2_svm:.4f} ({r2_svm*100:.2f}% de varianza explicada)")

print("\nReporte de Clasificación (SVM):")
print(classification_report(y_test, y_pred_svm))

In [ ]:
# ========================================================
# Creamos la tabla comparativa
# ========================================================
results = pd.DataFrame({
    'Modelo': ['Regresión Logística', 'Árbol de Decisión', 'Support Vector Machine'],
    'RMSE': [rmse_lr, rmse_tree, rmse_svm],
    'MAE': [mae_lr, mae_tree, mae_svm],
    'R²': [r2_lr, r2_tree, r2_svm]
})

# Ordenar por R² (descendente)
results = results.sort_values('R²', ascending=False)

print("\n" + " COMPARATIVA DE MODELOS ".center(50, "="))
print(results.to_string(index=False))

# Identificar mejor modelo
best_model = results.iloc[0]['Modelo']
best_r2 = results.iloc[0]['R²']

print(f"\n GANADOR: {best_model}")
print(f"   • Capacidad de Explicación (R²): {best_r2*100:.2f}%")
print(f"   • Error Medio (RMSE): {results.iloc[0]['RMSE']:.4f}")

In [ ]:
# ========================================================
# Comparación de modelos con validación cruzada
# ========================================================
print("\n" + "="*50)
print("COMPARACIÓN DE MODELOS CON VALIDACIÓN CRUZADA")
print("="*50)

models = {
    'Regresión Logística': LogisticRegression(random_state=42),
    'SVM': SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42),
    'Árbol de Decisión': DecisionTreeClassifier(max_depth=4, random_state=42)
}

# Lista para almacenar resultados
cv_results = []

for name, model in models.items():
    scores = cross_val_score(model, X_scaled, y, cv=5, scoring='accuracy')
    cv_results.append({
        'Modelo': name,
        'Accuracy Promedio': scores.mean(),
        'Desviación Estándar': scores.std(),
        'Scores': scores
    })
    print(f"{name}: Accuracy promedio = {scores.mean():.3f}, Desviación Estándar = {scores.std():.3f}")

# Visualización de resultados de validación cruzada
cv_df = pd.DataFrame(cv_results)[['Modelo', 'Accuracy Promedio', 'Desviación Estándar']]
print("\nResumen de validación cruzada:")
print(cv_df)

plt.figure(figsize=(10, 6))
sns.barplot(x='Modelo', y='Accuracy Promedio', data=cv_df, palette='viridis')
plt.errorbar(x=range(len(cv_df)), y=cv_df['Accuracy Promedio'],
             yerr=cv_df['Desviación Estándar'], fmt='none', color='black', capsize=5)
plt.title('Comparación de Modelos - Validación Cruzada (5-fold)')
plt.ylim(0.7, 1.0)  # Ajustar según los resultados
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig('cross_validation_comparison.png')
plt.show()

In [ ]:
# ========================================================
# Visualización de fronteras de decisión
# ========================================================
print("\n" + "="*50)
print("VISUALIZACIÓN DE FRONTERAS DE DECISIÓN")
print("="*50)

# Para visualizar las fronteras de decisión, usaremos solo Age y EstimatedSalary
print("Preparando visualización con 2 dimensiones (Age y EstimatedSalary)...")

# Entrenamos nuevos modelos con solo 2 características pero usando los mismos datos de entrenamiento
# Seleccionamos solo Age y EstimatedSalary del conjunto de entrenamiento
X_train_2d = X_train[:, -2:] if X_train.shape[1] > 2 else X_train

# Entrenamos modelos con solo 2 dimensiones para la visualización
lr_2d = LogisticRegression(random_state=42)
svm_2d = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
tree_2d = DecisionTreeClassifier(max_depth=4, random_state=42)

# Entrenamos estos modelos simplificados
lr_2d.fit(X_train_2d, y_train)
svm_2d.fit(X_train_2d, y_train)
tree_2d.fit(X_train_2d, y_train)

def plot_decision_boundary(model, X, y, title):
    h = 0.02  # Tamaño del paso en la malla
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # Predecir con el modelo
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, alpha=0.8, cmap=ListedColormap(['#FFAAAA', '#AAFFAA']))

    # Graficar los puntos de entrenamiento
    scatter = plt.scatter(X[:, 0], X[:, 1], c=y,
                         edgecolors='k', cmap=ListedColormap(['#FF0000', '#00FF00']))

    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title(title)
    plt.xlabel('Edad (estandarizada)')
    plt.ylabel('Salario estimado (estandarizado)')
    plt.legend(*scatter.legend_elements(), title="Compra")
    plt.savefig(f'decision_boundary_{title.replace(" ", "_").lower()}.png')
    plt.show()

# Obtenemos datos de X_scaled pero solo las últimas 2 columnas (Age y EstimatedSalary)
X_2d_vis = X_scaled[:, -2:] if X_scaled.shape[1] > 2 else X_scaled

# Visualizar fronteras de decisión para cada modelo (usando solo 2 dimensiones)
print("Generando visualizaciones de fronteras de decisión...")
plot_decision_boundary(lr_2d, X_2d_vis, y, "Regresión Logística")
plot_decision_boundary(svm_2d, X_2d_vis, y, "SVM")
plot_decision_boundary(tree_2d, X_2d_vis, y, "Árbol de Decisión")


In [ ]:
# ========================================================
# INTERPRETACIÓN DE RESULTADOS
# ========================================================
print("\n" + "="*50)
print("📊 INTERPRETACIÓN DE RESULTADOS 📊")
print("="*50)

print("Existe una fuerte relación entre los niveles elevados de compuestos orgánicos volátiles (VOC) detectados cerca de la máquina y la probabilidad de fallo.")
print("Además, se observa que cuando aumentan simultáneamente los índices de calidad del aire (AQ) y los niveles de VOC cerca de la máquina, es cuando mayor")
print("probabilidad de fallo se presenta. Esto sugiere que la acumulación de contaminantes en el entorno inmediato de la máquina es un indicador temprano de posibles fallos.")
print()

# --- Correlaciones Clave ---
print("CORRELACIONES CLAVE IDENTIFICADAS:")
print("-" * 70)
print(" • Compuestos Orgánicos (VOC) ↔ Fallas:                  Correlación = 0.797 (MUY ALTA)")
print(" • Calidad del Aire (AQ)  ↔ Compuestos Orgánicos (VOC):  Correlación = 0.619 (ALTA)")
print(" • Calidad del Aire (AQ)  ↔ Fallas:                      Correlación = 0.583 (ALTA)")
print()

# Métricas comparativas
print("METRICAS COMPARATIVAS:")
print("   • Los modelos muestran alto desempeño (>88% accuracy) por lo que se puede usar cualquier modelo")
print("   • El R² de 55.27% sugiere que existen otros factores no considerados en el modelo que también influyen en los fallos y que se deberian tomar en cuenta para el proximo planteamiento")
print("   • Error absoluto medio (MAE) muy bajo: 0.11-0.12 por lo que se puede confiar en los modelos")
print("   • El modelo SVM tiene una ligera ventaja en la identificación de casos positivos (fallos) por lo que se elegira para la implementacion")
print()

# Conclusión
print("CONCLUSIÓN FINAL:")
print("Comparando los modelos de clasificación (SVM, Árbol de Decisión y Regresión Logística) demostró que todos los algoritmos alcanzaron un desempeño similar y sólido, con una precisión")
print("promedio cercana al 90%, lo que confirma la robustez del enfoque predictivo para la detección de fallas. La fuerte correlación identificada entre los compuestos orgánicos volátiles")
print("(VOC) y el índice de calidad del aire (AQ) con las fallas de la máquina evidencia que estos sensores son indicadores críticos del estado operativo del equipo.")

print("Se recomienda implementar un sistema de monitoreo en tiempo real que integre estos dos sensores como variables predictoras principales y aunque se puede implementar cualquiera de ")
print("los 3 modelos es preferible SVM por su ligera ventaja en la especificidad.")

